## Importing

In [2]:
## %cd /Users/bezdek/Box/DCL_ARCHIVE/Documents/Events/exp148_Corpus/viz

import pickle as pkl
import pandas as pd
import numpy as np

# For panel visualizations:
import panel as pn
#import param
# For displaying images:
import cv2
from IPython.display import clear_output, Image, display, HTML
from scipy.spatial.distance import cosine
from scipy.ndimage import gaussian_filter1d
from glob import glob
import os
import sys
import matplotlib
import colorcet as cc

matplotlib.use('agg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from copy import deepcopy
#from run_sem_with_features import preprocess_skel
import joblib
from src.utils import get_point_biserial, get_binned_prediction
from src.utils import contain_substr
import seaborn as sns
from datetime import datetime


pn.extension()


### Initialize Variables and Widgets

In [6]:
%cd ..

C:\Users\nguye\Documents\PBS\Research\extended-event-modeling


In [10]:

df = pd.read_csv('output/run_sem/results_purity_coverage.csv')
old_path = '../extended-event-modeling-bk/extended-event-modeling/output/run_sem/results_purity_coverage.csv'
if os.path.exists(old_path):
    df_old = pd.read_csv(old_path)
    df = pd.concat([df, df_old], axis=0)
df = df[~df['tag'].isna()]
df = df[~df['bicorr'].isna()]
df = df.dropna(axis=0)

df['epoch'] = df['epoch'].astype(int)
df['chapter'] = df['run'].apply(lambda x: int(x[2]))
df['entropy_abs'] = df['entropy'] * np.log(df['n_event_models'])
df['config'] = df['tag'].apply(lambda tag: tag[:tag.find('grid')])
df['number_boundaries'] = df['number_boundaries'].clip(upper=400)


tags = glob(f'output/run_sem/aug*') + glob(f'output/run_sem/sep*')
tags = [t.replace('\\', '/') for t in tags]
tags = [x.split('/')[-1] for x in tags if any(i.isdigit() for i in x)]
available_tags = sorted(tags)
default_tag = available_tags[0]
available_metrics = sorted(list(df.columns))
default_metric = ['mean_pe', 'epoch', 'n_event_models', 'active_event_models', 'entropy', 'pearson_r', 'bicorr', 'percentile', 'number_boundaries',
                  'purity', 'coverage']
available_runs = sorted(list(df['run'].unique()))
default_run = available_runs
available_train_runs = list(df[(df['is_train'] == True)]['run'].unique())
default_train_run = available_train_runs
available_valid_runs = list(df[(df['is_train'] == False)]['run'].unique())
default_valid_run = available_valid_runs

multi_tag_select = pn.widgets.MultiSelect(name='Select 2 or 3 Tags for Comparison', value=[default_tag], options=available_tags,
                                          height=100, width=500)
multi_metric_select = pn.widgets.MultiSelect(name='Select Metrics to Visualize', value=default_metric, options=available_metrics,
                                             height=100)
multi_run_select = pn.widgets.MultiSelect(name='Select Runs to Visualize', value=default_run, options=available_runs,
                                          height=100)
multi_train_run_select = pn.widgets.MultiSelect(name='Select train_runs to Visualize', value=default_train_run,
                                                options=available_train_runs,
                                                height=100)
multi_valid_run_select = pn.widgets.MultiSelect(name='Select valid_runs to Visualize', value=default_valid_run,
                                                options=available_valid_runs,
                                                height=100)

train_checkbox = pn.widgets.Checkbox(name='Display Train?', value=True)
valid_checkbox = pn.widgets.Checkbox(name='Display Valid?', value=False)
compare_config = pn.widgets.Checkbox(name='Compare Configs?', value=False)

hues = ['tag', 'chapter', 'run', 'config', 'is_train', 'grain']
hue_select = pn.widgets.Select(name='Select Hue', options=hues, value='tag')

splits = ['tag', 'chapter', 'run', 'config', 'is_train', 'grain']
split_select = pn.widgets.Select(name='Select Split', options=hues, value='is_train')

min_epoch_slider = pn.widgets.DiscreteSlider(name='Select Min Epoch',
                                             options=list(range(df['epoch'].min(), df['epoch'].max()+1)), value=df['epoch'].min())
max_epoch_slider = pn.widgets.DiscreteSlider(name='Select Max Epoch',
                                             options=list(range(df['epoch'].min(), df['epoch'].max()+1)), value=df['epoch'].max())

In [37]:
re.findall("1[0-9][0-9]0", 'sdfsad_1000_1E-2')[0]

'1000'

In [30]:
alfas = sorted(list(set([x.split('_')[-2] for x in available_tags])))
lmdas = sorted(list(set([x.split('_')[-1] for x in available_tags])))
import re
tags_with_seed = [x for x in available_tags if len(re.findall("1[0-9][0-9]0", x))]
configs = sorted(list(set([x[:x.find(re.findall("1[0-9][0-9]0", x)[0])] for x in tags_with_seed])))

multi_alfa_select = pn.widgets.MultiSelect(name='Select Alfas', value=[alfas[0]], options=alfas)
multi_lmda_select = pn.widgets.MultiSelect(name='Select Lmdas', value=[lmdas[0]], options=lmdas)
multi_config_select = pn.widgets.MultiSelect(name='Select Configs', value=[configs[0]], options=configs)


@pn.depends(multi_alfa_select, multi_lmda_select, multi_config_select)
def alfa_lmda_to_tag(multi_alfa_select, multi_lmda_select, multi_config_select):
    select_tags = []
    for alfa in multi_alfa_select:
        for lmda in multi_lmda_select:
            if float(alfa[-5:]) == float(lmda[-5:]):
                continue
            for config in multi_config_select:
                select1 = f'{alfa}_{lmda}'
                select2 = f'{config}'
                # select_tags.extend([t for t in available_tags if select1 in t and select2 in t and select3 in t])
                select_tags.extend([t for t in available_tags if select1 in t and select2 in t])
    multi_tag_select.value = sorted(select_tags)

In [31]:
import param


class ActionExample(param.Parameterized):
    """
    Demonstrates how to use param.Action to trigger an update.
    """

    #     number = param.Number(default=0)

    action = param.Action(lambda x: x.param.trigger('action'), label='Update Data!')

    #     @param.depends('action')
    #     def get_number(self):
    #         return self.number
    def get_df_select(self):
        print(f'Update data: \n'
              f'tags={multi_tag_select.value} \n'
              f'runs={multi_run_select.value}')
        all_runs = multi_train_run_select.value + multi_valid_run_select.value
        df_select = df[(df['tag'].isin(multi_tag_select.value)) & (df['run'].isin(all_runs))
                       & (df['epoch'] <= max_epoch_slider.value) & (df['epoch'] >= min_epoch_slider.value)
                       ]
        if not (train_checkbox.value and valid_checkbox.value):
            if train_checkbox.value:
                df_select = df_select[(df_select['is_train'] == True)]
            elif valid_checkbox.value:
                df_select = df_select[(df_select['is_train'] == False)]
        return df_select

    action_matrix = param.Action(lambda x: x.param.trigger('action_matrix'), label='Update Matrix pane')
    @param.depends('action_matrix')
    def matrix_visualize(self):

        df_select = self.get_df_select()

        tags = sorted(list(df_select['tag'].unique()))
        print(f'Tags for plotting Matrix: {tags}')
        if len(tags) == 0:
            return None
        if valid_checkbox.value:
            alpha = 0.05
        else:
            alpha = 0.2
        hue = hue_select.value
        split = split_select.value
        split_order = sorted(list(df_select[split].unique()))
        hue_order = sorted(list(df_select[hue].unique()))

        print('Plotting Matrix pane...')
        # pairplot
#         sns_plot = sns.pairplot(df_select[multi_metric_select.value + [f'{hue}']],
#                                 hue=f'{hue}', hue_order=hue_order, palette='bright',
#                                 # kind='scatter', plot_kws={'alpha': alpha, 's': 10},
#                                 kind='reg', plot_kws={'scatter_kws': {'alpha': alpha, 's': 10}, 'ci': None, 'lowess': True},
# #                                 height=1.6,
#                                 x_vars=['epoch'], y_vars=multi_metric_select.value,
#                                )
#         for lh in sns_plot._legend.legendHandles:
#             lh._sizes = [100]
#         fig = sns_plot.fig

        # scatter plot or reg plot, with limitations for each
#         metrics = [x for x in multi_metric_select.value if x != 'epoch']
#         fig, axes = plt.subplots(nrows=len(metrics), ncols=len(split_order), squeeze=False, 
#                                  figsize=(6*len(split_order), 4*len(metrics)), sharex=True)
#         for i, metric in enumerate(metrics):
#             axes[i, 0].get_shared_y_axes().join(*axes[i, :])
#             for j, h in enumerate(split_order):
#                 sns.scatterplot(data=df_select[df_select[f'{split}'] == h], x='epoch', y=f'{metric}', ax=axes[i, j],
#                                 hue=f'{hue}', palette='deep', alpha=0.5,
#                                 )
# #                 sns.regplot(data=df_select[df_select[f'{split}'] == h], x='epoch', y=f'{metric}', ax=axes[i, j], 
# #                              lowess=True, scatter_kws={'alpha': 0.5},
# #                             )
#                 axes[i, j].set_title(f'{h}')

        # lmplot
        metrics = [x for x in multi_metric_select.value if x != 'epoch']
        imgs = []
        counter = 0
        
        for i, metric in enumerate(metrics):
            g = sns.lmplot(data=df_select, x='epoch', y=f'{metric}', col=f'{split}',
                           lowess=True, scatter_kws={'alpha': 0.3}, 
                           hue=f'{hue}', palette=cc.glasbey_dark, 
#                            legend = False,
#                            legend_out=True,
                           facet_kws={'legend_out': True},
                           )
            g.figure.savefig(f'tmp_{counter}.png')
            img = cv2.imread(f'tmp_{counter}.png')
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imgs.append(img)
#             imgs.append(cv2.cvtColor(np.asarray(g.figure.canvas.buffer_rgba()), cv2.COLOR_RGBA2BGR))
            
            plt.close(g.figure)
#         metrics = [x for x in multi_metric_select.value if x != 'epoch']
#         df_select = df_select.pivot_table(index=['epoch', f'{hue}'], values=metrics).stack().reset_index()
#         df_select.columns = ['epoch', f'{hue}', 'metric', 'value']
#         g = sns.FacetGrid(df_select, row='metric', col=f"{hue}", sharey="row", height=4, aspect=2)
#         g.map(sns.scatterplot, "epoch", 'value', alpha=.2)
#         fig = g.figure
        
#         fig.savefig('matrix.png')
        plt.close()
        print('Done Matrix pane!')
        imgs = cv2.vconcat(imgs)
        timestamp = datetime.now().strftime('matrix_%H_%M_%d_%m_%Y.png')
        cv2.imwrite(f"output/diagnose/matrix/{timestamp}", cv2.cvtColor(imgs, cv2.COLOR_RGB2BGR))
        from PIL import Image
        pil_image = Image.fromarray(imgs)
        return pil_image
    
    action_pe = param.Action(lambda x: x.param.trigger('action_pe'), label='Update PE pane')
    @param.depends('action_pe')
    def plot_pe(self):
        df_select = self.get_df_select()
        tags = sorted(list(df_select['tag'].unique()))
        if len(tags) == 0:
            return None
        if valid_checkbox.value:
            alpha = 0.3
        else:
            alpha = 0.5

        fig, axes = plt.subplots(nrows=1, ncols=len(tags), figsize=(4*len(tags), 4), squeeze=False, sharex=True, sharey=True)
        fig.suptitle(f'PE across epochs for all Tags', fontsize=14)
        axes = axes.reshape((1, len(tags)))
        print('Plot PE pane...')
        for tag, ax in zip(tags, axes[0]):
            df_tag = df_select[df_select['tag'] == tag]
            sns.regplot(x="epoch", y="mean_pe", data=df_tag, ax=ax, lowess=True, scatter_kws={'alpha': alpha})
            title = tag.split('grid')[0]
            ax.set_title(f'{title}')
        fig.savefig('pe_across_tags.png')
        plt.close()
        print('Done PE pane!')
        return fig



action_matrix = ActionExample()

In [32]:
dashboard = pn.Column(
    pn.Row('Compare Scatter Matrices Across Configurations'),
    pn.Row(multi_alfa_select, multi_lmda_select, multi_config_select),
    pn.Row(multi_metric_select, alfa_lmda_to_tag, multi_tag_select, multi_train_run_select, multi_valid_run_select),
    pn.Row(min_epoch_slider, max_epoch_slider),
    pn.Row(
            # action_matrix.param.action,
            train_checkbox,
            valid_checkbox,
            hue_select,
            split_select),
    pn.Column(action_matrix.param.action_matrix, action_matrix.matrix_visualize),
    pn.Column(action_matrix.param.action_pe, action_matrix.plot_pe),
)

# Launch the dashboard
dashboard.servable()

Update data: 
tags=['aug_05_n15_1030_1E-03_1E-01_1E+07'] 
runs=['1.1.10_kinect', '1.1.1_kinect', '1.1.2_kinect', '1.1.3_kinect', '1.1.4_kinect', '1.1.5_kinect', '1.1.6_kinect', '1.1.7_kinect', '1.1.8_kinect', '1.1.9_kinect', '1.2.10_kinect', '1.2.1_kinect', '1.2.3_kinect', '1.2.4_kinect', '1.2.6_kinect', '1.2.7_kinect', '1.2.8_kinect', '1.2.9_kinect', '1.3.10_kinect', '1.3.1_kinect', '1.3.3_kinect', '1.3.4_kinect', '1.3.5_kinect', '1.3.6_kinect', '1.3.7_kinect', '1.3.8_kinect', '1.3.9_kinect', '2.2.10_kinect', '2.2.1_kinect', '2.2.2_kinect', '2.2.3_kinect', '2.2.4_kinect', '2.2.5_kinect', '2.2.6_kinect', '2.2.7_kinect', '2.2.8_kinect', '2.2.9_kinect', '2.3.10_kinect', '2.3.1_kinect', '2.3.2_kinect', '2.3.3_kinect', '2.3.4_kinect', '2.3.5_kinect', '2.3.6_kinect', '2.3.7_kinect', '2.3.8_kinect', '2.3.9_kinect', '2.4.10_kinect', '2.4.1_kinect', '2.4.2_kinect', '2.4.3_kinect', '2.4.4_kinect', '2.4.5_kinect', '2.4.6_kinect', '2.4.7_kinect', '2.4.8_kinect', '2.4.9_kinect', '3.1.10_kinect', '

Column
    [0] Row
        [0] Markdown(str)
    [1] Row
        [0] MultiSelect(name='Select Alfas', options=['1E-01'], value=['1E-01'])
        [1] MultiSelect(name='Select Lmdas', options=['1E+07'], value=['1E+07'])
        [2] MultiSelect(name='Select Configs', options=['aug_05_n15_'], value=['aug_05_n15_'])
    [2] Row
        [0] MultiSelect(height=100, name='Select Metrics t..., options=['active_event_models', ...], value=['mean_pe', 'epoch', ...])
        [1] ParamFunction(function)
        [2] MultiSelect(height=100, name='Select 2 or 3..., options=['aug_05_n15_1030_1E-03_1E...], sizing_mode='fixed', value=['aug_05_n15_1030_1E-03_1E...], width=500)
        [3] MultiSelect(height=100, name='Select train_runs t..., options=['4.4.9_kinect', ...], value=['4.4.9_kinect', ...])
        [4] MultiSelect(height=100, name='Select valid_runs t..., options=['2.2.1_kinect', ...], value=['2.2.1_kinect', ...])
    [3] Row
        [0] DiscreteSlider(formatter='%d', name='Select Min Epoch', options=[1, 2, 3, 4, 5, ...], value=1)
        [1] DiscreteSlider(formatter='%d', name='Select Max Epoch', options=[1, 2, 3, 4, 5, ...], value=108)
    [4] Row
        [0] Checkbox(name='Display Train?', value=True)
        [1] Checkbox(name='Display Valid?')
        [2] Select(options=['tag', 'chapter', ...], value='tag')
        [3] Select(options=['tag', 'chapter', ...], value='is_train')
    [5] Column
        [0] Button(name='Update Matrix pane')
        [1] ParamMethod(method)
    [6] Column
        [0] Button(name='Update PE pane')
        [1] ParamMethod(method)